In [1]:
# %load main.py
import os
import re
import shutil
from tqdm import tqdm

import numpy  as np
import pandas as pd
import joblib
import nltk
import ekphrasis
from collections import Counter

### Step1: Data Pre-processing

In [2]:
# read data
Path =os.path.dirname(os.getcwd())
data_pathA=os.path.join(Path,'Datasets/A/twitter-2016train-A.txt')

In [3]:
data_pathA

'D:\\UCL-ELEC0135\\Assignments\\Datasets/A/twitter-2016train-A.txt'

In [4]:
# transform data into df form
dataA = pd.read_table(data_pathA,sep='\t',header=None)
dataA.columns = ['ID','Sentiment','Text','Nan']
def add_label(sentiment):
    if sentiment == 'negative':
        return 0
    elif sentiment == 'neutral':
        return 1
    elif sentiment == 'positive':
        return 2

dataA['label'] = dataA.Sentiment.apply(add_label)
dataA

,ID,Sentiment,Text,Nan,label
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...",NaN,1
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,NaN,1
2,619971047195045888,negative,If these runway renovations at the airport pre...,NaN,0
3,619974445185302528,neutral,If you could ask an onstage interview question...,NaN,1
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,NaN,2
...,...,...,...,...,...
20627,681877834982232064,neutral,@ShaquilleHoNeal from what I think you're aski...,NaN,1
20628,681879579129200640,positive,"Iran ranks 1st in liver surgeries, Allah bless...",NaN,2
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...,NaN,1
20630,681904976860327936,negative,@VanityFair Alex Kim Kardashian worth how to ...,NaN,0


In [5]:
# sentiment distribution of data
dataA.loc[:,['Sentiment','label']].value_counts().to_dict()

{('neutral', 1): 10342, ('positive', 2): 7059, ('negative', 0): 3231}

1Case conversion
包含“India”和“india”的语料库如果不应用小写化，机器会把它们识别为两个独立的术语，而实际上它们都是同一个单词的不同形式，并且对应于同一个国家。小写化后，仅存在一种“India”实例，即“india”，简化了在语料库中找到所有提到印度时的任务。

In [6]:
#import ekphrasis library
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

C:\Users\HCY\AppData\Roaming\Python\Python38\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\HCY\AppData\Roaming\Python\Python38\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [7]:
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop = set(stopwords.words('english'))

In [8]:
def Tokenize(Texts):
    token=[]
    for Text in Texts:
        words = [sentence for sentence in text_processor.pre_process_doc(Text) if (sentence!='s' and sentence!='\'')]
#         words = [word for word in words if (word not in stop)]
        token.append(words)
    words=[word for words in token for word in words]
    
    print("All words: {}".format(len(words)))
    # Create Counter
    counts = Counter(words)
    print("Unique words: {}".format(len(counts)))

    Most_common= counts.most_common()[:30]
    print("Top 30 most common words: {}".format(Most_common))
    
    vocab = {word: num for num, word in enumerate(counts, 1)}
    id2vocab = {v: k for k, v in vocab.items()}
    return token,vocab

In [9]:
token,vocab=Tokenize(dataA.Text)

All words: 493747
Unique words: 22054
Top 30 most common words: [('.', 18786), ('the', 17197), (',', 9728), ('to', 9144), ('i', 7607), ('<user>', 7027), ('<url>', 6377), ('in', 6203), ('a', 6176), ('and', 5986), ('on', 5528), ('<number>', 5262), ('of', 5162), ('<repeated>', 5142), ('<hashtag>', 5092), ('</hashtag>', 5092), ('is', 4868), ('for', 4392), ('!', 4391), ('<allcaps>', 4306), ('</allcaps>', 4306), ('it', 4213), ('you', 4037), ('not', 3590), ('may', 3563), ('-', 3550), ('with', 3397), ('be', 3357), ('tomorrow', 2953), ('?', 2787)]


### Step2: Word2Vec Pretraining

In [10]:
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
import multiprocessing
import tensorboard 

In [11]:
word2vec_model=Word2Vec(token,window=5, min_count=1,workers = multiprocessing.cpu_count())

In [12]:
word2vec_model.train(token, total_examples = len(token), epochs = 100)

(36803515, 49374700)

In [13]:
print('This is summary of Word2Vec: {}'.format(word2vec_model))

This is summary of Word2Vec: Word2Vec(vocab=22054, vector_size=100, alpha=0.025)


In [14]:
index=word2vec_model.wv.key_to_index

In [15]:
word2vec_model.wv.save_word2vec_format('Word2Vec.vector')

In [16]:
word_path=os.path.join(Path,'Datasets/datastories.twitter.100d.txt')

In [17]:
embed_matrix = np.zeros((len(index), 100))
embed_dict={}
for word, i in index.items():
    if word in word2vec_model.wv:
        embed_matrix[i] = word2vec_model.wv[word]
        embed_dict[word] = word2vec_model.wv[word]

In [18]:
del word2vec_model

In [19]:
# word = pd.read_table(word_path,sep=' ',header=None)
# word.set_index(0,inplace=True)
# Embed_dict={}
# for i in range(word.shape[0]):
#     Embed_dict[word.index[i]]=word.iloc[i,:]

In [20]:
# Embed_path=os.path.join(Path,'Datasets/Embed_dict')


### Step3: Training

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SpatialDropout1D, Bidirectional

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import datetime
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from IPython import display
import transformers
import tensorflow_hub as hub
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
display.set_matplotlib_formats('svg')

C:\Users\HCY\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version:  2.6.0
Eager mode:  True
GPU is available


C:\Users\HCY\AppData\Local\Temp\ipykernel_28292\281481831.py:23: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [22]:
import tensorflow_addons as tfa

In [23]:
def tokenizer_lstm(X, vocab, seq_len):
    '''
    Returns tokenized tensor with left/right padding at the specified sequence length
    '''
    X_tmp = np.zeros((len(X), seq_len), dtype=np.int64)
    for i, text in enumerate(X):
        tokens = [word for word in text_processor.pre_process_doc(text) if (word!='s' and word!='\'')]
#         tokens = [word for word in tokens if (word not in stop)]
        token_ids = [vocab[word] for word in tokens if word in embed_dict.keys()]###
        end_idx = min(len(token_ids), seq_len)
        start_idx = max(seq_len - len(token_ids), 0)
        X_tmp[i,start_idx:] = token_ids[:end_idx]

    return X_tmp

In [24]:
X=tokenizer_lstm(dataA.Text, vocab, 100)###

In [25]:
Y = tf.one_hot(dataA.label, depth=3)

In [26]:
Y= np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size=0.1, random_state=1000) 

In [27]:
def build_embedding_layer(vocab, embed_dict):
    """
    Build embedding matrix and embedding layer
    :param vocab_size: vocabulary size
    :param tok: tokenizer
    :param embed_dict: embedding index
    :return: embedding matrix and embedding layer
    """
    #Build embedding matrix
    vocab_size=len(vocab)+1
    embedding_matrix = np.zeros((vocab_size, 100))
    for word, i in vocab.items():
        # Vector corresponds to word
        embedding_vector = embed_dict.get(word)###,embed_dict['<unk>']

        if embedding_vector is not None:
            # Ensure vector of embedding_matrix row matches word index
            embedding_matrix[i] = embedding_vector
            
    # Build embedding layer
    embedding_layer = Embedding(input_dim = vocab_size, output_dim = 100, weights = [embedding_matrix], input_length = 100, trainable=False)
    return embedding_layer

In [28]:
# def build_embedding_layer(vocab, embeddings_index):
#     """
#     Build embedding matrix and embedding layer
#     :param vocab_size: vocabulary size
#     :param tok: tokenizer
#     :param embeddings_index: embedding index
#     :return: embedding matrix and embedding layer
#     """
#     #Build embedding matrix
#     vocab_size=len(vocab)+1
#     embedding_matrix = np.zeros((vocab_size, 100))
#     for word, i in vocab.items():
#         try:
#             # Vector corresponds to word
#             embedding_vector = embeddings_index.get(word)
#         except:
#             embedding_vector = embeddings_index['<unk>']#['unknown']#['<unk>']
#         if embedding_vector is not None:
#             # Ensure vector of embedding_matrix row matches word index
#             embedding_matrix[i] = embedding_vector
#     # Build embedding layer
#     embedding_layer = Embedding(input_dim = vocab_size, output_dim = 100, weights = [embedding_matrix], input_length = 100, trainable=False)
#     return embedding_layer,embedding_matrix

In [29]:
embedding_layer=build_embedding_layer(vocab, embed_dict) ###

In [30]:
# del embed_dict,embed_matrix#, Embed_dict,embedding_matrix,

In [50]:
def model_train(X_train, y_train, embedding_layer):
        """
        Train, validate and test BiLSTM model, calculate accuracy of training and validation set
        :param X_train: tweet train data
        :param y_train: sentiment label train data
        :param embedding_layer: embedding layer
        :param X_test: tweet test data
        :param y_test: sentiment label test data
        :return: accuracy, recall, precision, F1 score and history
        """
        tf.debugging.set_log_device_placement(True)
        model = Sequential()
        model.add(embedding_layer)
        model.add(SpatialDropout1D(0.2))
        
#         LSTM(128, dropout = 0.2, recurrent_dropout = 0.5)

#         LSTM(128,activation='tanh', recurrent_activation='sigmoid',
#              use_bias=True,dropout=0.5,recurrent_dropout=0.0)
    
#         model.add(Bidirectional(LSTM(128,dropout = 0.5,return_sequences=True)))
#         model.add(Bidirectional(LSTM(64,dropout = 0.5)))   27  loss: 0.7183 - accuracy: 0.6815 - val_loss: 0.7699 - val_accuracy: 0.6634


#         model.add(Bidirectional(LSTM(128,dropout = 0.5))) 26 loss: 0.7517 - accuracy: 0.6596 - val_loss: 0.7827 - val_accuracy: 0.6532
        
#         model.add(Bidirectional(LSTM(128,dropout = 0.5,return_sequences=True)))
#         model.add(Bidirectional(LSTM(128,dropout = 0.0)))   28 loss: 0.6897 - accuracy: 0.6925 - val_loss: 0.7774 - val_accuracy: 0.6610

#         model.add(Bidirectional(LSTM(128,dropout = 0.5,return_sequences=True)))
#         model.add(Bidirectional(LSTM(64,dropout = 0.5)))  29 loss: 0.7080 - accuracy: 0.6759 - val_loss: 0.7618 - val_accuracy: 0.6653

        model.add(Bidirectional(LSTM(128,dropout = 0.5,return_sequences=True)))
        model.add(Bidirectional(LSTM(64,dropout = 0.5)))  #27 loss: 0.6735 - accuracy: 0.7028 - val_loss: 0.7640 - val_accuracy: 0.6669

#         model.add(Bidirectional(LSTM(128,dropout = 0.3,recurrent_dropout = 0.5)))
    
        model.add(Dense(3, activation = 'softmax'))
        model.summary()
        
        batch_size = 128
        epochs = 50

        model.compile(optimizer='adam', loss='categorical_crossentropy', 
                      metrics = ['Recall','Accuracy','Precision'])
        history = model.fit(X_train, y_train, validation_split = 0.2, epochs = epochs, batch_size = batch_size)
        model.save('taskA.h5'.format(25))
        train_acc = history.history['Accuracy'][-1]
        val_acc = history.history['val_Accuracy'][-1]
        
        train_rec = history.history['recall'][-1]
        val_rec = history.history['val_recall'][-1]
        
        train_pre = history.history['precision'][-1]
        val_pre = history.history['val_precision'][-1]
        return train_acc, val_acc, train_rec, val_rec, train_pre, val_pre,history

In [51]:
train_acc, val_acc, train_rec, val_rec, train_pre, val_pre,history= model_train(X_train, Y_train, embedding_layer)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          2205500   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 100, 100)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 100, 256)          234496    
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 2,604,735
Trainable params: 399,235
Non-trainable params: 2,205,500
_________________________________________________________________
Epoch 1/50
117/117 [==============================] - 9s 45ms/step - loss: 0.9203 - recall: 0.3862 - Acc

Epoch 33/50
117/117 [==============================] - 4s 35ms/step - loss: 0.6699 - recall: 0.6476 - Accuracy: 0.6983 - precision: 0.7313 - val_loss: 0.7796 - val_recall: 0.6252 - val_Accuracy: 0.6572 - val_precision: 0.6791
Epoch 34/50
117/117 [==============================] - 4s 35ms/step - loss: 0.6606 - recall: 0.6517 - Accuracy: 0.7029 - precision: 0.7344 - val_loss: 0.7913 - val_recall: 0.6449 - val_Accuracy: 0.6688 - val_precision: 0.6789
Epoch 35/50
117/117 [==============================] - 4s 35ms/step - loss: 0.6615 - recall: 0.6483 - Accuracy: 0.7024 - precision: 0.7366 - val_loss: 0.7751 - val_recall: 0.6360 - val_Accuracy: 0.6599 - val_precision: 0.6813
Epoch 36/50
117/117 [==============================] - 4s 35ms/step - loss: 0.6644 - recall: 0.6517 - Accuracy: 0.7012 - precision: 0.7362 - val_loss: 0.7898 - val_recall: 0.6322 - val_Accuracy: 0.6616 - val_precision: 0.6788
Epoch 37/50
117/117 [==============================] - 4s 34ms/step - loss: 0.6574 - recall: 0.6

In [ ]:
class_names = ['0: Negative','1: Neutra;', '2: Positive']

model = load_model('taskA.h5')
Y_pred = model.predict(X_test)
Metrcs = model.evaluate(X_test, Y_test, return_dict=True)

In [ ]:
Y_pred

In [ ]:
def pred_eval(model):
    Y_pred = model.predict(X_test)
    Metrcs = model.evaluate(X_test, Y_test, return_dict=True)
    
    return